In [1]:
import re

import pandas as pd

import nltk

from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline #make_pipeline, make_union

from sklearn.model_selection import train_test_split
# 

from collections import Counter

In [3]:
df_instances = pd.read_json("../data/instances.jsonl", lines=True);
df_truth = pd.read_json("../data/truth.jsonl", lines=True); 

# Display first 5 rows of both
display(df_instances.head())
display(df_truth.head())

,id,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle
0,608310377143799808,[],[Apple's iOS 9 'App thinning' feature will giv...,Tue Jun 09 16:31:10 +0000 2015,['App thinning' will be supported on Apple's i...,'App thinning' will be supported on Apple's iO...,"Apple,gives,gigabytes,iOS,9,app,thinning,featu...",[Paying for a 64GB phone only to discover that...,Apple gives back gigabytes: iOS 9 'app thinnin...
1,609297109095972864,[media/609297109095972864.jpg],[RT @kenbrown12: Emerging market investors are...,Fri Jun 12 09:52:05 +0000 2015,"[Stocks Fall as Investors Watch Central Banks,...",Global investors have yanked $9.3 billion from...,"emerging market,emerging markets,em flows,em i...","[Emerging markets are out of favor., Global in...",Emerging Markets Suffer Largest Outflow in Sev...
2,609504474621612032,[],[U.S. Soccer should start answering tough ques...,Fri Jun 12 23:36:05 +0000 2015,[US to vote for Ali in FIFA election and not B...,A U.S. Senator's scathing letter questioned U....,,"[WINNIPEG, Manitoba â€“ The bubble U.S. Soccer...",U.S. Soccer should start answering tough quest...
3,609748367049105408,[],[How theme parks like Disney World left the mi...,Sat Jun 13 15:45:13 +0000 2015,"[Some 1,000 persons turned out in Albuquerque,...","America's top family vacation spots, like the ...","disney, disney world, disney ticket prices, di...",[When Walt Disney World opened in an Orlando s...,How theme parks like Disney World left the mid...
4,608688782821453824,[media/608688782821453825.jpg],[Could light bulbs hurt your health? One compa...,Wed Jun 10 17:34:49 +0000 2015,[Electric lights have made the world safer and...,One company will put a health notice on all th...,"health, Should there be warning labels on your...",[(CNN)The light bulb always makes the world's ...,Warning labels on your light bulbs


,id,truthClass,truthJudgments,truthMean,truthMedian,truthMode
0,608310377143799808,no-clickbait,"[0.0, 0.6666667, 0.0, 0.33333334000000003, 0.0]",0.200000,0.000000,0.000000
1,609297109095972864,no-clickbait,"[0.6666667, 0.0, 0.0, 0.0, 0.0]",0.133333,0.000000,0.000000
2,609504474621612032,clickbait,"[0.33333334000000003, 0.6666667, 1.0, 0.0, 0.6...",0.533333,0.666667,0.666667
3,609748367049105408,no-clickbait,"[1.0, 0.0, 0.33333334000000003, 0.333333340000...",0.466667,0.333333,0.333333
4,608688782821453824,clickbait,"[1.0, 0.33333334000000003, 0.6666667, 0.333333...",0.666667,0.666667,1.000000


In [10]:
df_truth.groupby('truthClass').count()

,id,truthJudgments,truthMean,truthMedian,truthMode
truthClass,,,,,
clickbait,762,762,762,762,762
no-clickbait,1697,1697,1697,1697,1697


In [36]:
is_number_regex = r"[0-9]+"

def extract_features_titles(df):
    def extract_from_title(title):
        result = dict()
        tiny = title.strip().lower()
        title_words = nltk.word_tokenize(tiny)
        title_words_number_repl = [re.sub(is_number_regex, "[n]", word) for word in title_words]
        twrr_bigram_count = Counter(nltk.bigrams(title_words_number_repl))
        result.update({'title_bigram[{}]'.format(bigram): count for bigram, count in twrr_bigram_count.items()})
        return result
    return map(extract_from_title, df['targetTitle'])

In [34]:
list(extract_features_titles(df_instances.iloc[:5]['targetTitle']))

apple gives back gigabytes: ios 9 'app thinning' feature will finally give your phone's storage a boost
emerging markets suffer largest outflow in seven years
u.s. soccer should start answering tough questions about hope solo
how theme parks like disney world left the middle class behind
warning labels on your light bulbs


[{"title_bigram[('apple', 'gives')]": 1,
  "title_bigram[('gives', 'back')]": 1,
  "title_bigram[('back', 'gigabytes')]": 1,
  "title_bigram[('gigabytes', ':')]": 1,
  "title_bigram[(':', 'ios')]": 1,
  "title_bigram[('ios', '[n]')]": 1,
  'title_bigram[(\'[n]\', "\'app")]': 1,
  'title_bigram[("\'app", \'thinning\')]': 1,
  'title_bigram[(\'thinning\', "\'")]': 1,
  'title_bigram[("\'", \'feature\')]': 1,
  "title_bigram[('feature', 'will')]": 1,
  "title_bigram[('will', 'finally')]": 1,
  "title_bigram[('finally', 'give')]": 1,
  "title_bigram[('give', 'your')]": 1,
  "title_bigram[('your', 'phone')]": 1,
  'title_bigram[(\'phone\', "\'s")]': 1,
  'title_bigram[("\'s", \'storage\')]': 1,
  "title_bigram[('storage', 'a')]": 1,
  "title_bigram[('a', 'boost')]": 1},
 {"title_bigram[('emerging', 'markets')]": 1,
  "title_bigram[('markets', 'suffer')]": 1,
  "title_bigram[('suffer', 'largest')]": 1,
  "title_bigram[('largest', 'outflow')]": 1,
  "title_bigram[('outflow', 'in')]": 1,
  "ti

In [53]:
clickbaitClassifier = Pipeline([
    ('feature_extraction', FunctionTransformer(extract_features_titles, validate=False)),
    ('encoder', DictVectorizer()),
    ('classifier', DecisionTreeClassifier())
])

In [46]:
# Do a split
merged = pd.merge(df_instances, df_truth, on='id')
X_train, X_test, y_train, y_test = train_test_split(merged, merged['truthClass'])
display(X_train.head())
display(X_test.head())

,id,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,truthClass,truthJudgments,truthMean,truthMedian,truthMode
1665,608865805200183296,[],[Two Aussies play horrible prank on their frie...,Thu Jun 11 05:18:14 +0000 2015,[2016%2f09%2f16%2fee%2fhttpsd2mhye01h4nj2n.clo...,A couple of serial pranksters have crossed an ...,"australia, uncategorized, watercooler, videos,...","[Too far?, A couple of serial pranksters have ...",Two Aussies play horrible prank on their frien...,clickbait,"[0.0, 0.6666667, 0.6666667, 1.0, 0.0]",0.466667,0.666667,0.666667
2251,608008142916452352,[media/608008142916452353.jpg],[Avian flu taking a bite out of #McMuffin avai...,Mon Jun 08 20:30:12 +0000 2015,[Most of McDonald's most popular morning sandw...,Grocers across the country have already starte...,,[As the latest avian flu epidemic continues to...,Avian flu taking a bite out of McMuffin availa...,clickbait,"[1.0, 1.0, 0.6666667, 0.6666667, 0.33333334000...",0.733333,0.666667,1.000000
2071,609102862308810752,[media/609102862308810753.jpg],[Bryce Dallas Howard cries on command talking ...,Thu Jun 11 21:00:13 +0000 2015,[],"There, there, Bryce. It'll be okay. We'll get ...","entertainment-brief,bryce dallad howard,conan,...","[There, there, Bryce. It'll be okay. We'll get...",Bryce Dallas Howard Cries On Command While Tal...,no-clickbait,"[1.0, 0.0, 0.33333334000000003, 0.333333340000...",0.333333,0.333333,0.000000
2295,609417442893262848,[media/609417442893262848.jpg],[VIDEO: Sounders reserve player gets red card ...,Fri Jun 12 17:50:15 +0000 2015,[],Although some red cards are questionable or...,"World Football, MLS, Breaking News",[Although some red cards are questionable or c...,Sounders Reserve Player Gets Red Card for Kick...,no-clickbait,"[0.33333334000000003, 0.0, 0.0, 0.0, 0.3333333...",0.133333,0.000000,0.000000
322,609348021697048576,[],[Britainâ€™s already fragile union is at risk ...,Fri Jun 12 13:14:24 +0000 2015,[Ben Jennings illlustration â€“ a British bull...,With the Conservatives trading in narrow natio...,"Scotland,Scottish National party (SNP),Politic...","[If the United Kingdom collapses, it will not ...",Britainâ€™s already fragile union is at risk -...,clickbait,"[1.0, 1.0, 0.33333334000000003, 0.0, 0.6666667]",0.600000,0.666667,1.000000


,id,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle,truthClass,truthJudgments,truthMean,truthMedian,truthMode
211,608859210269470720,[media/608859210269470720.jpg],[Keep Fidgeting! Movement helps improve focus ...,Thu Jun 11 04:52:02 +0000 2015,[],"Fidgeting in the classroom may be disruptive, ...",,"[USA, New Jersey, Jersey City, Girl (8-9) doin...",Keep Fidgeting! Movement Helps Improve Focus i...,no-clickbait,"[0.33333334000000003, 0.0, 0.33333334000000003...",0.200000,0.333333,0.333333
1277,609371231289233408,[],[Elizabeth Banks tapping Red Queen for vein of...,Fri Jun 12 14:46:37 +0000 2015,[Elizabeth Banks.],The actor and Pitch Perfect 2 director might a...,"Elizabeth Banks,Science fiction and fantasy,Cu...",[The actor and Pitch Perfect 2 director might ...,Elizabeth Banks tapping Red Queen for vein of ...,clickbait,"[0.6666667, 0.33333334000000003, 0.0, 1.0, 0.6...",0.533333,0.666667,0.666667
1606,608717216758964224,[media/608717216758964224.jpg],[The Angels drafted the brother of Mike Troutâ...,Wed Jun 10 19:27:48 +0000 2015,"[ANAHEIM, CA - JUNE 01: Mike Trout #27 of the...",And with the 585th pick of the 2015 MLB dra...,"Baseball, MLB, AL West, Los Angeles Angels, Br...",[And with the 585th pick of the 2015 MLB draft...,Los Angeles Angels of Anaheim Draft Mike Trout...,clickbait,"[0.6666667, 0.6666667, 0.0, 0.3333333400000000...",0.466667,0.666667,0.666667
2352,609009344257138688,[media/609009344257138688.jpg],[American killed fighting against ISIS hailed ...,Thu Jun 11 14:48:37 +0000 2015,[],The family of Keith Broomfield says he was awa...,,"[In a video made before his death, Keith Broom...","Keith Broomfield, American Slain Fighting ISIS...",no-clickbait,"[0.0, 0.33333334000000003, 0.33333334000000003...",0.400000,0.333333,0.333333
1273,610174056575463424,[media/610174056575463424.png],[Prince George centre of attention at family o...,Sun Jun 14 19:56:46 +0000 2015,[Catherine Duchess of Cambridge and Prince Geo...,The relaxed afternoon was a world away from th...,"prince george, william, kate, duchess, royal, ...",[Clutching his toy car and playing happily in ...,Prince George kicks a ball and plays with toy ...,no-clickbait,"[0.0, 0.33333334000000003, 0.33333334000000003...",0.200000,0.333333,0.333333


In [54]:
clickbaitClassifier.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('feature_extraction', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function extract_features_titles at 0x000001E7B2941EA0>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)), ('encoder', DictVectorizer...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

In [55]:
clickbaitClassifier.score(X_test, y_test)

0.5284552845528455

In [56]:
tr = clickbaitClassifier.named_steps['classifier']
dv = clickbaitClassifier.named_steps['encoder']
dfFeatureImportance = pd.DataFrame(list(zip(dv.feature_names_, tr.feature_importances_)))
dfFeatureImportance.sort_values(1, ascending=False)

,0,1
1560,"title_bigram[('[n]', 'things')]",0.008639
16102,"title_bigram[('ways', 'to')]",0.005716
14346,"title_bigram[('the', 'best')]",0.005666
16873,"title_bigram[('you', 'have')]",0.004786
8171,"title_bigram[('in', 'the')]",0.004133
10887,"title_bigram[('of', 'thrones')]",0.004119
7981,"title_bigram[('in', 'america')]",0.003876
9395,"title_bigram[('look', 'at')]",0.003874
682,"title_bigram[('(', 'and')]",0.003871
1180,"title_bigram[(':', 'everything')]",0.003858
